In [1]:
import mysql.connector
import pandas as pd
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="your_password",
    database="main"
)
#i put a placeholder in password

In [2]:
def get_select_query_results(connessione, query):
    try:
        # Creazione dell'oggetto cursor
        cursor = connessione.cursor()

        # Esecuzione della query
        cursor.execute(query)
        colonne = [col[0] for col in cursor.description]
        # Recupero dei risultati
        risultati = cursor.fetchall()

        # Chiusura del cursore
        cursor.close()

        # Restituzione dei risultati
        return pd.DataFrame(risultati, columns = colonne)

    except mysql.connector.Error as err:
        # Gestione degli errori
        print(f"Errore MySQL: {err}")
        return None
    
    except:
        cursor.close()
        return None

##### Query 1: TOP 5 Champions to play in support role, rank GOLD

In [3]:
query_1 = """
    SELECT position, champion_name as Champion, grade, games_played as GamesPlayed
    FROM game_stats
    JOIN champion ON game_stats.champion_ID = champion.champion_ID
    JOIN lane ON game_stats.lane_ID = lane.lane_ID
    JOIN ranking on game_stats.ranking_ID = ranking.ranking_ID
    WHERE lane_name = 'support' AND ranking_name = 'gold'
    AND position BETWEEN 1 AND 5
    ORDER BY position ASC
"""
get_select_query_results(conn, query_1)


,position,Champion,grade,GamesPlayed
0,1,Senna,1.111,306700
1,2,Xerath,1.111,198432
2,3,Lux,1.222,428303
3,4,Blitzcrank,1.222,226385
4,5,Zyra,1.222,195733


#### Query 2: How to Counter Senna, in support role, ranking Gold


In [4]:
query_2 = """
    SELECT vs_champion.champion_name as Counter, winrate, games_played as GamesPlayed
    FROM matchup
    JOIN lane ON matchup.lane_ID = lane.lane_ID
    join ranking on matchup.ranking_ID = ranking.ranking_ID
    JOIN champion on matchup.champion_ID = champion.champion_ID
    join champion as vs_champion on matchup.champion_vs_ID = vs_champion.champion_ID
    where ranking_name = 'gold' and champion.champion_name = 'Senna' and lane_name = 'support' and type = 'counter'
    ORDER BY winrate ASC
"""

get_select_query_results(conn, query_2)

,Counter,winrate,GamesPlayed
0,Zyra,45.78,5100
1,Xerath,46.38,4644
2,Vel''Koz,47.05,1526
3,Amumu,47.36,492
4,Lux,47.44,10302


Query 3: Average difficulty and grade of META Champions (top 5) for every lane in bronze ranking

In [5]:
query_3 = """
    SELECT lane.lane_name as Lane, AVG(champion_base_info.difficulty) as AVGDiff, AVG(game_stats.grade) as AVGGrade
    FROM game_stats
    JOIN ranking ON game_stats.ranking_ID = ranking.ranking_ID
    JOIN lane on game_stats.lane_ID = lane.lane_ID
    JOIN champion_base_info on game_stats.champion_ID = champion_base_info.champion_ID
    WHERE (position BETWEEN 1 AND 5) AND ranking_name = 'bronze'
    group by lane.lane_name

"""
get_select_query_results(conn, query_3)


,Lane,AVGDiff,AVGGrade
0,jungle,3.8000,1.288800
1,adc,5.2000,1.499867
2,mid,6.0000,1.344333
3,top,5.6000,1.199867
4,support,5.8000,1.066533


Query 4: Considering all rankings, who are the top 2 champions for every role?


In [6]:
query_4 = """
    SELECT lane_name as Lane, position, champion_name as Champion, grade
    FROM game_stats
    JOIN ranking ON  game_stats.ranking_ID = ranking.ranking_ID
    JOIN champion on game_stats.champion_ID = champion.champion_ID
    JOIN lane on game_stats.lane_ID = lane.lane_ID
    where (game_stats.position BETWEEN 1 and 2) AND ranking_name = 'all' 
    ORDER BY game_stats.lane_ID ASC, position ASC

"""

get_select_query_results(conn, query_4)

,Lane,position,Champion,grade
0,top,1,Jax,0.888667
1,top,2,Illaoi,1.111000
2,jungle,1,Briar,0.888667
3,jungle,2,Graves,1.000000
4,mid,1,Sylas,0.888667
5,mid,2,Fizz,1.111000
6,adc,1,Vayne,1.000000
7,adc,2,Jhin,1.111000
8,support,1,Rakan,0.888667
9,support,2,Xerath,1.111000


Query 5: who are the champions whose grade < 1, considering all rankings?(Overpowered Champions)

In [7]:
query_5 = """
    select champion_name as Champion, lane_name as Lane, grade
    from game_stats
    join champion on game_stats.champion_ID  = champion.champion_ID
    join lane on game_stats.lane_ID = lane.lane_ID
    join ranking on game_stats.ranking_ID  = ranking.ranking_ID
    where ranking_name = 'all' and grade < 1
"""

get_select_query_results(conn, query_5)

,Champion,Lane,grade
0,Jax,top,0.888667
1,Briar,jungle,0.888667
2,Rakan,support,0.888667
3,Sylas,mid,0.888667


The lack of ADC champions with a grade < 1 underscores the ongoing challenge of effectively balancing this crucial role in League of Legends.